In [0]:
!pip install -q tensorflow
!pip install -q sklearn
!pip install -q torch
!pip install -q pretrainedmodels
!pip install -q pandas

In [0]:
%matplotlib inline
import tensorflow as tf
import glob
import numpy as np
from PIL import Image
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#import pretrainedmodels
import torch
#import pretrainedmodels.utils as utils
import numpy as np
from keras.preprocessing import sequence
import os
import tensorflow as tf
import glob
import pandas as pd
from sklearn.preprocessing import StandardScaler
from PIL import Image
from sklearn.preprocessing import minmax_scale

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

############################################Encoder#################################################
#model_name = 'vgg16' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
#print(pretrainedmodels.pretrained_settings[model_name])
#model.eval()

# with open('./8/8/8.txt') as f:
#     content = [x.strip('\n') for x in f.readlines()]

# print(content[0].split("|")[0])
# str1='./8/train_8/'

data_input=[]
data_output=[]

for i in range(0,len(content)):
    try:
        str2=content[i].split("|")[0]
        str3=str1+str2
        #print(str3)
        load_img = utils.LoadImage()

        # transformations depending on the model
        # rescale, center crop, normalize, and others (ex: ToBGR, ToRange255)
        tf_img = utils.TransformImage(model)

        path_img = str3

        input_img = load_img(path_img)
        input_tensor = tf_img(input_img)  
        input_tensor = input_tensor.unsqueeze(0)
        input = torch.autograd.Variable(input_tensor,
                                        requires_grad=False)

        features = model.features(input)
        features = torch.Tensor.detach(features).numpy()
        data_input.append(features[0])
        capt=content[i].split("|")[2].lstrip()
        data_output.append(capt)

        if(i%100==0):
            print(i)

        if(i%1000==0):
          np.savez('data_input_vgg.npz', name1=data_input)
          np.savez('data_output_vgg.npz', name1=data_output)
    except:
        pass

#Normalization
# data_input=np.array(data_input)
# print(data_input)
# for i in range (0,4096):
#    data_input[:,i]=minmax_scale(data_input[:,i])

# print(data_input[0])
# print(data_input[1])
#data_input=np.load('/content/gdrive/My Drive/feats.npy','r')

data_input=np.load('/content/gdrive/My Drive/data_input_goog_train_3.npz')['name1']
data_output=np.load('/content/gdrive/My Drive/data_output_vgg.npz')['name1']

data_input_test=np.load('/content/gdrive/My Drive/data_input_goog_test_3.npz')['name1']
data_output_test=np.load('/content/gdrive/My Drive/data_output_vgg_test.npz')['name1']

feats_test=data_input_test
captions_test=data_output_test

feats_tot=data_input
captions_tot=data_output

feats=feats_tot
captions=captions_tot

print(data_output[0])

l# print(data_input[10],data_output[10])
# print(data_input1[10],data_output1[10])
# print(len(data_input),len(data_input[0]))
# print(len(data_output),len(data_output[0]))

############################################################Decoder################################

model_path = '/content/gdrive/My Drive/models/tensorflow'

#################Processing words###############
def preProBuildWordVocab(sentence_iterator, word_count_threshold=15): 
    print('preprocessing %d word vocab' % (word_count_threshold, ))
    word_counts = {}
    nsents = 0
    for sent in sentence_iterator:
      nsents += 1
      for w in sent.lower().split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1
    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

    ixtoword = {}
    ixtoword[0] = '.'
    wordtoix = {}
    wordtoix['#START#'] = 0
    ix = 1
    for w in vocab:
      wordtoix[w] = ix
      ixtoword[ix] = w
      ix += 1

    word_counts['.'] = nsents
    bias_init_vector = np.array([1.0*word_counts[ixtoword[i]] for i in ixtoword])
    bias_init_vector /= np.sum(bias_init_vector)
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector)
    return wordtoix, ixtoword, bias_init_vector.astype(np.float32)


class Captions():
    
    def initialize_param(self,init_b):
        with tf.device('/cpu:0'):
            self.word_embedding = tf.Variable(tf.random_uniform([self.n_words, self.dim_embed], -0.1, 0.1))

            self.embedding_bias = tf.Variable(tf.zeros([dim_embed]))
            self.lstm = tf.contrib.rnn.BasicLSTMCell(dim_hidden)

            self.img_embedding = tf.Variable(tf.random_uniform([dim_in, dim_hidden], -0.1, 0.1))
            self.img_embedding_bias = tf.Variable(tf.zeros([dim_hidden]))

            self.word_encoding = tf.Variable(tf.random_uniform([dim_hidden, self.n_words], -0.1, 0.1))
            
            if init_b is not None:
              self.word_encoding_bias = tf.Variable(init_b)
            else:
              self.word_encoding_bias = tf.Variable(tf.zeros([n_words]))
 

    def __init__(self,dim_in, dim_embed, dim_hidden, batch_size,steps, n_words, init_b):

        self.dim_in = dim_in
        self.dim_embed = dim_embed
        self.dim_hidden = dim_hidden
        self.batch_size = batch_size
        self.n_lstm_steps = steps
        self.n_words = n_words

        self.initialize_param(init_b)
        
    def build_model(self):
            caption_placeholder = tf.placeholder(tf.int32, [self.batch_size, self.n_lstm_steps])
            mask = tf.placeholder(tf.float32, [self.batch_size, self.n_lstm_steps])

            img = tf.placeholder(tf.float32, [self.batch_size, self.dim_in])

            image_embedding = tf.matmul(img, self.img_embedding) + self.img_embedding_bias

            state = self.lstm.zero_state(self.batch_size, dtype=tf.float32)

            total_loss = 0.0
            
            with tf.variable_scope("RNN"):
                i=0
                while(i<self.n_lstm_steps):
                    if i > 0:
                        with tf.device('/cpu:0'):
                          current_embedding = tf.nn.embedding_lookup(self.word_embedding,caption_placeholder[:, i - 1]) + self.embedding_bias
                          tf.get_variable_scope().reuse_variables()
                    else:
                        current_embedding = image_embedding
                        
                    out, state = self.lstm(current_embedding, state)

                    if i > 0:
                        labels = tf.expand_dims(caption_placeholder[:, i], 1)
                        ix_range = tf.range(0, self.batch_size, 1)
                        ixs = tf.expand_dims(ix_range, 1)
                        concat = tf.concat([ixs, labels], 1)
                        onehot = tf.sparse_to_dense(
                            concat, tf.stack([self.batch_size, self.n_words]), 1.0, 0.0)

                        logit = tf.matmul(out, self.word_encoding) + self.word_encoding_bias
                        xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=onehot)
                        xentropy = xentropy * mask[:, i]

                        loss = tf.reduce_sum(xentropy)
                        total_loss += loss
                    
                    i=i+1

                total_loss = total_loss / tf.reduce_sum(mask[:, 1:])
                return total_loss, img, caption_placeholder, mask

    def build_generator(self, maxlen, batchsize=1):
            
            img = tf.placeholder(tf.float32, [self.batch_size, self.dim_in])
            image_embedding = tf.matmul(img, self.img_embedding) + self.img_embedding_bias
            state = self.lstm.zero_state(batchsize, dtype=tf.float32)

            all_words = []
            with tf.variable_scope("RNN"):
                output, state = self.lstm(image_embedding, state)
                previous_word = tf.nn.embedding_lookup(self.word_embedding, [0]) + self.embedding_bias

                i=0
                while(i<maxlen):
                    tf.get_variable_scope().reuse_variables()

                    out, state = self.lstm(previous_word, state)

                    logit = tf.matmul(out, self.word_encoding) + self.word_encoding_bias
                    best_word = tf.argmax(logit, 1)

                    previous_word = tf.nn.embedding_lookup(self.word_embedding, best_word)
                    previous_word += self.embedding_bias

                    all_words.append(best_word)
                    i=i+1

            return img, all_words

####################Training RNN##################
dim_embed = 256
dim_hidden = 256
dim_in = 2048
batch_size = 128
momentum = 0.9
n_epochs = 100


def train(learning_rate=0.001, continue_training=False, transfer=True):
    tf.reset_default_graph()

    #feats=data_input
    #captions=data_output
    #print(captions[0].lstrip())
    wordtoix, ixtoword, init_b = preProBuildWordVocab(captions)
    np.save('ixtoword.npy', ixtoword)

    sess = tf.InteractiveSession()
    n_words = len(wordtoix)
    maxlen = np.max( [x for x in map(lambda x: len(x.split(' ')), captions) ] )
    caption_generator = Captions(dim_in, dim_hidden, dim_embed, batch_size, maxlen+2, n_words, init_b)

    loss, image, sentence, mask = caption_generator.build_model()
    saver = tf.train.Saver(max_to_keep=100)
    # print(loss)
    # print(sentence)
    # print(mask)
    # print(image)
    # print(maxlen)
    # #np.save('data/ixtoword', ixtoword)

    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(learning_rate, global_step,int(len(feats) / batch_size), 0.95)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    tf.global_variables_initializer().run()

    losses = []
    for epoch in range(n_epochs):
        for start, end in zip(range(0, len(feats), batch_size), range(batch_size, len(feats), batch_size)):

            current_feats = feats[start:end]
            current_captions = captions[start:end]
            current_caption_ind = [x for x in map(lambda cap: [wordtoix[word] for word in cap.lower().split(' ')[:-1] if word in wordtoix],current_captions)]

            current_caption_matrix = sequence.pad_sequences(current_caption_ind, padding='post', maxlen=maxlen + 1)
            current_caption_matrix = np.hstack([np.full((len(current_caption_matrix), 1), 0), current_caption_matrix])

            current_mask_matrix = np.zeros((current_caption_matrix.shape[0], current_caption_matrix.shape[1]))
            nonzeros = np.array([x for x in map(lambda x: (x != 0).sum() + 2, current_caption_matrix)])

            for ind, row in enumerate(current_mask_matrix):
                row[:nonzeros[ind]] = 1

            _, loss_value = sess.run([train_op, loss], feed_dict={
                image: current_feats.astype(np.float32),
                sentence: current_caption_matrix.astype(np.int32),
                mask: current_mask_matrix.astype(np.float32)
            })

        print("Avg loss: ", loss_value, "\t Epoch {}/{}".format(epoch, n_epochs))
        saver.save(sess, os.path.join(model_path, 'model'), global_step=epoch)
        if(loss_value<=2.1):
            break

        #print("Saving the model from epoch: ", epoch)





A man sitting in a racing car with black sunglasses and a look that reads " I 'm ready to win . "


In [0]:
train(0.001,False,False)

preprocessing 15 word vocab
preprocessed words 14136 -> 2708


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Avg loss:  3.480961 	 Epoch 0/100
Avg loss:  3.1828113 	 Epoch 1/100
Avg loss:  3.0210776 	 Epoch 2/100
Avg loss:  2.9261062 	 Epoch 3/100
Avg loss:  2.8503425 	 Epoch 4/100
Avg loss:  2.785673 	 Epoch 5/100
Avg loss:  2.744398 	 Epoch 6/100
Avg loss:  2.6691751 	 Epoch 7/100
Avg loss:  2.6439848 	 Epoch 8/100
Avg loss:  2.61354 	 Epoch 9/100
Avg loss:  2.5650616 	 Epoch 10/100
Avg loss:  2.5374012 	 Epoch 11/100
Avg loss:  2.507603 	 Epoch 12/100
Avg loss:  2.4665754 	 Epoch 13/100
Avg loss:  2.4413068 	 Epoch 14/100
Avg loss:  2.403728 	 Epoch 15/100
Avg loss:  2.392152 	 Epoch 16/100
Avg loss:  2.3591044 	 Epoch 17/100
Avg loss:  2.3358681 	 Epoch 18/100
Avg loss:  2.3263166 	 Epoch 19/100
Avg loss:  2.2848492 	 Epoch 20/100
Avg loss:  2.2757018 	 Epoch 21/100
Avg loss:  2.2558494 	 Epoch 22/100
Avg loss:  2.2297387 	 Epoch 23/100
Avg loss:  2.205616 	 Epoch 24/100
Avg loss:  2.1905503 	 Epoch 25/100
Avg loss:  2.1868587 	 Epoch 26/100
Avg loss:  2.151852 	 Epoch 27/100
Avg loss:  2

In [0]:
  dim_embed = 256
  dim_hidden = 256
  dim_in = 2048
  batch_size = 1
  learning_rate = 0.001
  momentum = 0.9
  #n_epochs = 25
  wordtoix, ixtoword, init_b = preProBuildWordVocab(captions)
  np.save('ixtoword.npy', ixtoword)
  
  n_words = len(ixtoword)
  maxlen=29
   
  tf.reset_default_graph()
  sess = tf.InteractiveSession()
   
  caption_generator = Captions(dim_in, dim_hidden, dim_embed, batch_size, maxlen+2, n_words,None)

  image, generated_words = caption_generator.build_generator(maxlen=maxlen)

preprocessing 15 word vocab
preprocessed words 14136 -> 2708


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
import numpy as np

predicted_captions=[]
  
#data_input=np.load('/content/gdrive/My Drive/data_input_vgg.npz')['name1']
#data_output=np.load('/content/gdrive/My Drive/data_output_vgg.npz')['name1']

data_input_test=np.load('/content/gdrive/My Drive/data_input_goog_train_3.npz')['name1']
data_output_test=np.load('/content/gdrive/My Drive/data_output_vgg.npz')['name1']

feats=data_input_test
captions=data_output_test

# data_input=np.load('/content/gdrive/My Drive/feats.npy','r')
# #data_output=np.load('/content/gdrive/My Drive/data_output_vgg.npz')['name1']

# feats=data_input
# #captions=data_output

# annotation_path='/content/gdrive/My Drive/results_20130124.token'
# annotations = pd.read_table(annotation_path, sep='\t', header=None, names=['image', 'caption'])
# data_output=annotations['caption'].values
# captions=data_output
    
saver = tf.train.Saver()
saved_path=tf.train.latest_checkpoint(model_path)
saver.restore(sess, saved_path)

def test(sess,image,generated_words,ixtoword,idx):

    feat = np.array([feats[idx]])
    sanity_check= False

    generated_word_index= sess.run(generated_words, feed_dict={image:feat})
    generated_word_index = np.hstack(generated_word_index)

    generated_sentence = [ixtoword[x] for x in generated_word_index]
    #print(generated_sentence)
    sen=[]
    for j in range(0,len(generated_sentence)):
      if(generated_sentence[j]!='.'):
        sen.append(generated_sentence[j])
      if(generated_sentence[j]=='.'):
        break
    predicted_captions.append(sen)

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/models/tensorflow/model-30


In [0]:
#predicted_captions=[]
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu

data_input_test=np.load('/content/gdrive/My Drive/data_input_goog_train_3.npz')['name1']
data_output_test=np.load('/content/gdrive/My Drive/data_output_vgg.npz')['name1']

feats_test=data_input_test
captions_test=data_output_test

for i in range(0,len(feats_test)):
  test(sess,image,generated_words,ixtoword,i)
  if(i%1000==0):
    print(i)

capt=[]
pred=[]
c=0

for i in range(0,len(feats_test)-5,5):
  li=[]
  for j in range(i,i+5):
     li.append(captions_test[j].split())
  capt.append(li)
  pred.append(predicted_captions[i])


print(capt[2])
print(pred[2])
# print(len(capt))
# print(len(pred))
print('BLEU-1: %f' % corpus_bleu(capt, pred, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(capt, pred, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(capt, pred, weights=(0.3, 0.3, 0.3, 0)))
print('BLEU-4: %f' % corpus_bleu(capt, pred, weights=(0.25, 0.25, 0.25, 0.25)))

#print(captions[0].split())
#print(captions[15])

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
[['A', 'woman', 'in', 'a', 'red', 'dress', 'is', 'singing', 'on', 'a', 'stage', 'next', 'to', 'three', 'men', 'playing', 'instruments', '.'], ['three', 'men', 'dressed', 'in', 'blue', 'play', 'guitar', 'while', 'a', 'woman', 'sings', '.'], ['A', 'music', 'group', 'performs', 'while', 'dressed', 'up', '.'], ['a', 'band', 'performing', 'at', 'a', 'show', '.'], ['A', 'band', 'performing', 'on', 'a', 'stage']]
['a', 'band', 'performs', 'on', 'stage']
BLEU-1: 0.487316
BLEU-2: 0.301985
BLEU-3: 0.218208
BLEU-4: 0.111466


In [0]:
print(capt[1000])
print(pred[1000])

#Train
#2
#1000
#996
#10590


#Test
#0
#10
#25
#200
#1000

[['A', 'man', 'sits', 'on', 'a', 'bench', 'beside', 'his', 'bike', 'while', 'viewing', 'the', 'scenery', 'around', 'him', 'overlooking', 'a', 'busy', 'city', '.'], ['An', 'overweight', 'man', 'in', 'shorts', 'is', 'sitting', 'on', 'a', 'bench', 'next', 'to', 'a', 'bicycle', '.'], ['A', 'man', 'is', 'sitting', 'on', 'a', 'bench', 'with', 'his', 'bike', 'outside', 'of', 'the', 'city', '.'], ['A', 'man', 'sitting', 'on', 'a', 'bench', 'next', 'to', 'a', 'bicycle', '.'], ['Man', 'sitting', 'on', 'bench', 'next', 'to', 'bike', '.']]
['a', 'man', 'in', 'a', 'white', 'shirt', 'and', 'jeans', 'is', 'walking', 'down', 'a', 'sidewalk']
